In [8]:
# Install necessary packages: aiohttp for async subprocess execution and pyngrok for Ngrok integration
!pip install aiohttp pyngrok

import asyncio
import os

# Set LD_LIBRARY_PATH to prioritize system NVIDIA libraries over built-in ones
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
print(NGROK_AUTH_TOKEN)
# Define an async helper function to run commands asynchronously
async def run(cmd):
    print('>>> starting', *cmd)
    p = await asyncio.subprocess.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # Asynchronously process and print the output and error streams
    async def pipe(lines):
        async for line in lines:
            print(line.strip().decode('utf-8'))

    await asyncio.gather(
        pipe(p.stdout),
        pipe(p.stderr),
    )

# Authenticate with Ngrok using the token
await run(['ngrok', 'config', 'add-authtoken', NGROK_AUTH_TOKEN])

2lXglqF5Wu6BvUprTU4SwoTNw1x_65YUurGZ7zdpow85pkduB
>>> starting ngrok config add-authtoken 2lXglqF5Wu6BvUprTU4SwoTNw1x_65YUurGZ7zdpow85pkduB
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# Run multiple tasks concurrently:
# 1. Start the Ollama server.
# 2. Start Ngrok to forward HTTP traffic from the local Ollama API running on localhost:11434.
await asyncio.gather(
    run(['ollama', 'serve']),
    run(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header', 'localhost:11434']),
    # Uncomment the next line and replace with your Ngrok domain if using a static URL
    # run(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header', 'localhost:11434', '--domain', 'insert-your-static-ngrok-domain-here']),
)

>>> starting ollama serve
>>> starting ngrok http --log stderr 11434 --host-header localhost:11434
2024/09/03 11:44:44 routes.go:1125: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"
time=2024-09-03T11:44:44.606Z level=INFO source=images.go:753 msg="total blobs: 0"
time=2024-09-03T11:44:44

In [ ]:
# test api from local machine
# python code

import requests

# URL for the API endpoint
url = 'https://5c85-34-16-0-60.ngrok-free.app/api/generate'

# Function to make the POST request
def make_request(prompt):
    data = {
        "model": "llama2",
        "prompt": prompt,
        "stream": False
    }
    response = requests.post(url, json=data)
    if response.status_code == 200:
        return response.json().get("response", "No response received")
    else:
        return f"Error: {response.status_code}, {response.text}"

# Continuous chat loop
while True:
    # Get user input for the prompt
    prompt = input("You: ")

    # Check if the user wants to exit
    if prompt.lower() == 'exit':
        print("Exiting chat.")
        break

    # Make the request
    response = make_request(prompt)

    # Print the response
    print("Bot:", response)

In [ ]:
!ollama

Usage:
  ollama [flags]
  ollama [command]

Available Commands:
  serve       Start ollama
  create      Create a model from a Modelfile
  show        Show information for a model
  run         Run a model
  pull        Pull a model from a registry
  push        Push a model to a registry
  list        List models
  ps          List running models
  cp          Copy a model
  rm          Remove a model
  help        Help about any command

Flags:
  -h, --help      help for ollama
  -v, --version   Show version information

Use "ollama [command] --help" for more information about a command.
